# Clustering Crypto

In [ ]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import json

### Fetching Cryptocurrency Data

In [1]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

AttributeError: 'str' object has no attribute 'json'

In [2]:
# Create a DataFrame 
crypto_df = pd.DataFrame(json_data['Data']).T
crypto_df.head()

NameError: name 'pd' is not defined

In [1]:
# Alternatively, use the provided csv file:
file_path = Path("crypto_data.csv")

# Create a DataFrame
crypto_currency_df = pd.read_csv(file_path)

NameError: name 'Path' is not defined

### Data Preprocessing

In [ ]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df.drop(columns=['Rating','TotalCoinsFreeFloat', 'Url', 'Taxonomy', 'SortOrder', 'Sponsored', 'SmartContractAddress', 'Id', 'BuiltOn', 'FullName', 'ImageUrl', 'ContentCreatedOn', 'PreMinedValue'])

In [ ]:
# Keep only cryptocurrencies that are trading
crypto_df = rypto_df.loc[slim_crypto_df['IsTrading'] == True]


In [ ]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = slim_crypto_df.loc[slim_crypto_df['Algorithm'] != 'N/A']

In [ ]:
# Remove the "IsTrading" column
crypto_df = slim_crypto_df.drop(columns='IsTrading')

In [ ]:
# Remove rows with at least 1 null value
crypto_df = slim_crypto_df.dropna()

In [ ]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = slim_crypto_df.loc[slim_crypto_df['TotalCoinsMined'] != 0]

In [ ]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[crypto_df['TotalCoinSupply'] != 'N/A']
crypto_df = crypto_df.loc[crypto_df['ProofType'] != 'N/A']

In [ ]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
crypto_name = crypto_df

In [ ]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = slim_crypto_df.drop(columns='CoinName')

In [ ]:
# Create dummy variables for text features
crypto_dummy = pd.get_dummies(data=slim_crypto_df, columns=['Name', 'Algorithm', 'ProofType', 'Symbol'])


In [ ]:
# Standardize data
crypto_df_scaled = StandardScaler().fit_transform(slim_crypto_encoded)
crypto_df_scaled

### Reducing Dimensions Using PCA

In [ ]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_scaled)

In [ ]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=slim_crypto_df.index
)
pca_df.head()

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [ ]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Visualization", xlabel="Number of Clusters", ylabel="Inertia")

Running K-Means with `k=<your best value for k here>`

In [ ]:
# Initialize the K-Means model
k_mean_model = KMeans(n_clusters=6, random_state=0)
# Fit the model
k_mean_model.fit(pca_df)
# Predict clusters
predictions = k_mean_model.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "CoinName": names.CoinName,
    "Algorithm": slim_crypto_df.Algorithm,
    "ProofType": slim_crypto_df.ProofType,
    "TotalCoinsMined": slim_crypto_df.TotalCoinsMined,
    "TotalCoinSupply": slim_crypto_df.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "Class": kmodel.labels_,
    "Predictions": predictions, 
    },
)
clustered_df.head()

### Visualizing Results

#### 3D-Scatter with Clusters

In [ ]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']


In [ ]:
# Print the total number of tradable cryptocurrencies
clustered_df.hvplot.table(columns)

#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000000

In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="Inferno_r", 
    hover_cols=['CoinName']
)